In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [68]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [69]:
resultados = {}

In [70]:
def evaluar_modelo(nombre, y_true, y_pred):
    resultados[nombre] = {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision (0)': precision_score(y_true, y_pred, pos_label=0),
        'Precision (1)': precision_score(y_true, y_pred, pos_label=1),
        'Recall (0)': recall_score(y_true, y_pred, pos_label=0),
        'Recall (1)': recall_score(y_true, y_pred, pos_label=1),
        'F1 (0)': f1_score(y_true, y_pred, pos_label=0),
        'F1 (1)': f1_score(y_true, y_pred, pos_label=1),
    }

1. RandomForestClassifier 

In [71]:
from sklearn.ensemble import RandomForestClassifier

In [72]:
df_encoded = pd.read_csv('./data/dataset_encoded.csv')
df_encoded.head()

,Age,Patients number per hour,NRS_pain,SBP,HR,RR,BT,Group_1,Group_2,Sex_1,...,Arrival mode_5,Arrival mode_6,Arrival mode_7,Injury_1,Injury_2,Mental_1,Mental_2,Mental_3,Mental_4,target_emergencia
0,71,3,2.0,160.0,84.0,18.0,36.6,False,True,False,...,False,False,False,False,True,True,False,False,False,0
1,56,12,2.0,137.0,60.0,20.0,36.5,True,False,True,...,False,False,False,False,True,True,False,False,False,0
2,68,8,2.0,130.0,102.0,20.0,36.6,False,True,True,...,False,False,False,False,True,True,False,False,False,0
3,71,8,3.0,139.0,88.0,20.0,36.5,True,False,False,...,False,False,False,True,False,True,False,False,False,0
4,58,4,3.0,91.0,93.0,18.0,36.5,True,False,False,...,False,False,False,True,False,True,False,False,False,0


In [73]:
X = df_encoded.drop(columns='target_emergencia')
y = df_encoded['target_emergencia']

In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [79]:
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

In [80]:
print("RandomForest:")
print(classification_report(y_test, y_pred_rf))
evaluar_modelo('RandomForest', y_test, y_pred_rf)

RandomForest:
              precision    recall  f1-score   support

           0       0.70      0.60      0.65       134
           1       0.73      0.81      0.77       183

    accuracy                           0.72       317
   macro avg       0.72      0.71      0.71       317
weighted avg       0.72      0.72      0.72       317



2. XGBoostClassifier

In [82]:
from xgboost import XGBClassifier

In [83]:
xgb = XGBClassifier(eval_metric='logloss', random_state=42)
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

In [84]:
print("XGBoost:")
print(classification_report(y_test, y_pred_xgb))
evaluar_modelo('XGBoost', y_test, y_pred_xgb)

XGBoost:
              precision    recall  f1-score   support

           0       0.69      0.60      0.64       134
           1       0.73      0.80      0.76       183

    accuracy                           0.72       317
   macro avg       0.71      0.70      0.70       317
weighted avg       0.71      0.72      0.71       317



3. CatBoostClassifier

In [89]:
from catboost import CatBoostClassifier

In [90]:
df_selected = pd.read_csv('./data/dataset_selected.csv')
cat_features = ['Group', 'Sex', 'Arrival mode', 'Injury', 'Mental']

In [91]:
X = df_selected.drop(columns='target_emergencia')
y = df_selected['target_emergencia']

In [92]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [93]:
catboost = CatBoostClassifier(verbose=0, random_state=42)
catboost.fit(X_train, y_train, cat_features=cat_features)
y_pred_catboost = catboost.predict(X_test)

In [95]:
print("CatBoost:")
print(classification_report(y_test, y_pred_catboost))
evaluar_modelo('CatBoost', y_test, y_pred_catboost)

CatBoost:
              precision    recall  f1-score   support

           0       0.71      0.59      0.64       134
           1       0.73      0.82      0.77       183

    accuracy                           0.72       317
   macro avg       0.72      0.70      0.71       317
weighted avg       0.72      0.72      0.72       317



4. SVM (RBF)

In [107]:
from sklearn.svm import SVC

In [108]:
df_preprocess = pd.read_csv('./data/dataset_preprocess.csv')
X = df_preprocess.drop(columns='target_emergencia')
y = df_preprocess['target_emergencia']

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [110]:
svm = SVC(kernel='rbf', probability=True, random_state=42)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

In [111]:
print("SVM:")
print(classification_report(y_test, y_pred_svm))
evaluar_modelo('SVM', y_test, y_pred_svm)

SVM:
              precision    recall  f1-score   support

           0       0.70      0.51      0.59       134
           1       0.70      0.84      0.76       183

    accuracy                           0.70       317
   macro avg       0.70      0.67      0.68       317
weighted avg       0.70      0.70      0.69       317



5. MLPClassifier (Red neuronal)

In [112]:
from sklearn.neural_network import MLPClassifier

In [113]:
mlp = MLPClassifier(
    hidden_layer_sizes=(100,),
    max_iter=1000,
    early_stopping=True,
    random_state=42
)

In [114]:
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)

In [115]:
print("MLPClassifier:")
print(classification_report(y_test, y_pred_mlp))
evaluar_modelo('MLPClassifier', y_test, y_pred_mlp)

MLPClassifier:
              precision    recall  f1-score   support

           0       0.63      0.49      0.55       134
           1       0.68      0.79      0.73       183

    accuracy                           0.66       317
   macro avg       0.65      0.64      0.64       317
weighted avg       0.66      0.66      0.65       317



6. KNNClassifier

In [117]:
from sklearn.neighbors import KNeighborsClassifier

In [118]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_test)

In [119]:
print("KNN:")
print(classification_report(y_test, y_pred_knn))
evaluar_modelo('KNN', y_test, y_pred_knn)

KNN:
              precision    recall  f1-score   support

           0       0.60      0.64      0.62       134
           1       0.72      0.69      0.71       183

    accuracy                           0.67       317
   macro avg       0.66      0.67      0.66       317
weighted avg       0.67      0.67      0.67       317



In [121]:
df_resultados = pd.DataFrame(resultados).T.round(3)
display(df_resultados)

,Accuracy,Precision (0),Precision (1),Recall (0),Recall (1),F1 (0),F1 (1)
RandomForest,0.722,0.702,0.734,0.597,0.814,0.645,0.772
XGBoost,0.716,0.686,0.734,0.604,0.798,0.643,0.764
CatBoost,0.722,0.705,0.732,0.590,0.820,0.642,0.773
SVM,0.700,0.701,0.700,0.507,0.842,0.589,0.764
MLPClassifier,0.662,0.629,0.679,0.493,0.787,0.552,0.729
KNN,0.669,0.601,0.724,0.642,0.689,0.621,0.706


## Métricas clave según el objetivo

### Detectar emergencias sin errores graves
- **Métrica clave:** `Recall (1)`
- Mide qué tan bien el modelo identifica correctamente a los pacientes que realmente están en emergencia.
- Es fundamental en contextos clínicos donde omitir una emergencia (falso negativo) puede tener consecuencias graves.
- **Complemento útil:** `F1 (1)` para evaluar el balance general entre precisión y recall en la clase 1.

### Evitar saturar el hospital con falsos positivos
- **Métrica clave:** `Precision (1)`
- Evalúa qué tan confiables son las predicciones positivas del modelo.
- Es decir, de todos los pacientes que el modelo clasifica como "emergencia", ¿cuántos realmente lo son?
- **Complemento útil:** `Recall (0)` para medir qué tan bien identifica a los pacientes que no están en emergencia.

### Buscar equilibrio general del sistema
- **Métricas clave:** `F1-score promedio` y `Accuracy`
- Buscamos un rendimiento balanceado entre emergencias y no emergencias.
- Ideal para sistemas en etapas de prototipo o modelos de apoyo clínico donde ambos tipos de error deben mantenerse bajo control.


---

## Escenarios de aplicación y modelos recomendados

### Escenario 1: Hospital pequeño con recursos limitados
- **Prioridad:** minimizar la clasificación incorrecta de emergencias (falsos positivos).
- **Objetivo:** alta `Precision (1)` y buen `Recall (0)` para asegurar una asignación eficiente de recursos limitados.
- **Modelos recomendados:** `XGBoost` o `RandomForest`.

### Escenario 2: Sistema nacional de atención de emergencias (tipo SAMU)
- **Prioridad:** no dejar pasar ninguna emergencia real.
- **Objetivo:** alto `Recall (1)` aunque aumenten los falsos positivos.
- **Modelo recomendado:** `CatBoost`, que mostró el mejor `Recall (1)` en la evaluación (`0.820`).

### Escenario 3: Prototipo para derivación automatizada o triage digital
- **Prioridad:** construir un sistema base balanceado para revisión y ajuste posterior.
- **Objetivo:** buenas métricas generales (`Accuracy`, `F1 (1)`, `F1 (0)`).
- **Modelos recomendados:** `RandomForest` o `CatBoost`.
